In [2]:
import os
import sys
#list all folders in /scratch/groups/rondror/marvinli/combind_fragment/benchmark_set_split_no_copy
import glob
from combind_find import screen, get_scores 
import pandas as pd
#check if it is in the slurm

#running this on ondemand will fail
if 'SLURM_JOB_ID' in os.environ:
    sys.path.append('/scratch/groups/rondror/marvinli/combind_fragment/')
    features_screen_path = '/scratch/groups/rondror/marvinli/combind_fragment/benchmark_set_split_no_copy/*/features_screen/*'
else:
    sys.path.append('/home/pc/Documents/combind_fragment/combind_fragment/')
    features_screen_path = '/home/pc/Documents/combind_fragment/combind_fragment/benchmark_set_split_no_copy/*/features_screen/*'

features_screen_path = glob.glob(features_screen_path)


In [3]:
#extract the folder name before "features_screen"
from collections import defaultdict
from tqdm import tqdm

combind_protein_energy_dict = defaultdict(dict)
glide_protein_energy_dict = defaultdict(dict)

for feature_root in tqdm(features_screen_path):
    library_name = feature_root.split("/")[-1]
    protein_name = feature_root.split("/features_screen")[0]
    protein_name = protein_name.split("/")[-1]
    docking_file = feature_root.replace('features_screen', 'docking')

    docking_file = docking_file + "-*/*.maegz"
    docking_file = glob.glob(docking_file)[0]
    #stats_root = "/home/pc/Documents/combind_fragment/combind_fragment/stats_data/fragment_refit"
    stats_root = "/home/pc/Documents/combind_fragment/combind_fragment/stats_data/fragment_refit_prev"
    alpha = 1 #same as the default
    try:
        combind_energy = screen(None, feature_root, stats_root, alpha, 'hbond,saltbridge,contact')
    except:
        print(f"Error for {protein_name}")
        continue
    combind_energy_to_ligand, glide_energy_to_ligand = get_scores(docking_file, combind_energy)

    #get the best score for each ligand, for glide, the lowest score is the best, for combind, the highest score is the best
    best_combind_energy = {k: max(v) for k, v in combind_energy_to_ligand.items()}
    best_glide_energy = {k: min(v) for k, v in glide_energy_to_ligand.items()}
    
    #make dir
    os.makedirs(f"rank_csv/{protein_name}", exist_ok=True)
    #save to a csv for each protein
    pd.Series(best_combind_energy, name="combind_scores").to_csv(f"rank_csv/{protein_name}/combind_scores_{library_name}.csv")
    pd.Series(best_glide_energy, name="glide_scores").to_csv(f"rank_csv/{protein_name}/glide_scores_{library_name}.csv")
    
    combind_protein_energy_dict[protein_name].update(best_combind_energy)
    glide_protein_energy_dict[protein_name].update(best_glide_energy)
    
    

  6%|▌         | 8/129 [00:08<02:13,  1.10s/it]

Error for keap1


 70%|██████▉   | 90/129 [01:22<00:32,  1.18it/s]

In [4]:
#for each protein, find the best score
#save to a csv for each protein
for protein_name, scores in combind_protein_energy_dict.items():
    pd.Series(scores, name="combind_scores").to_csv(f"rank_csv/{protein_name}_combind_scores.csv")
    pd.Series(glide_protein_energy_dict[protein_name], name="glide_scores").to_csv(f"rank_csv/{protein_name}_glide_scores.csv")



In [76]:
protein_name = features_screen_path[0].split("/features_screen")[0]
protein_name = protein_name.split("/")[-1]
feature_root = features_screen_path[1]
docking_file = feature_root.replace('features_screen', 'docking')
#change to docking_file
docking_file = docking_file + "*/*.maegz"
docking_file = glob.glob(docking_file)[0]
stats_root = "/home/pc/Documents/combind_fragment/combind_fragment/stats_data/fragment_refit"
#stats_root = "/home/pc/Documents/combind_fragment/combind_fragment/stats_data/default"
alpha = 1 #same as the default
combind_energy = screen(features_screen_path, feature_root, stats_root, alpha, 'hbond,saltbridge,contact')

In [77]:
#import defaultdict from collections


In [81]:
combind_energy_to_ligand, glide_energy_to_ligand = get_scores(docking_file, combind_energy)

#get the best score for each ligand, for glide, the lowest score is the best, for combind, the highest score is the best
best_combind_energy = {k: max(v) for k, v in combind_energy_to_ligand.items()}
best_glide_energy = {k: min(v) for k, v in glide_energy_to_ligand.items()}


In [85]:
best_glide_energy

{'DEC_0952': -9.44308317069109,
 'DEC_0821': -9.00564699719285,
 'DEC_0877': -8.67502481272795,
 'DEC_0823': -8.50418334589371,
 'DEC_0975': -8.43510073150066,
 'DEC_0846': -8.38313897328121,
 'DEC_0976': -8.26669218298437,
 'DEC_0853': -8.25748885568494,
 'DEC_0797': -8.25523026705674,
 'DEC_0937': -8.24014966643033,
 'DEC_0980': -8.19318267289387,
 'DEC_0951': -8.17398295829422,
 'DEC_0862': -8.16308486076166,
 'DEC_0847': -8.13760676315408,
 'DEC_0950': -8.09984779624695,
 'DEC_0966': -8.07971415145579,
 'DEC_0834': -7.93651103622568,
 'DEC_0925': -7.78479227341507,
 'DEC_0964': -7.78168436445766,
 'DEC_0861': -7.75959895150942,
 'DEC_0874': -7.60620446280752,
 'DEC_0948': -7.5914365152939,
 'DEC_0928': -7.48600036753265,
 'DEC_0958': -7.46669296855597,
 'DEC_0903': -7.42974709101126,
 'DEC_0807': -7.40029909792837,
 'DEC_0851': -7.34270478625839,
 'DEC_0893': -7.29449651742157,
 'DEC_0849': -7.23427208085922,
 'DEC_0873': -7.20493836565171,
 'DEC_0868': -7.07086489566427,
 'DEC_083